# 7. Przetwarzanie obrazów. Sieci konwolucyjne

### Tomasz Rodak

Laboratorium 7

---

## 7.1 

[*Konwolucyjne sieci neuronowe*](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN) to sieci neuronowe typu feedforward, które rozpoznają cechy lokalne w danych wejściowych poprzez optymalizację *filtrów konwolucyjnych*. Sieci te można stosować do różnych typów danych, ale najczęściej wykorzystuje się je do rozpoznawania i przetwarzania obrazów.  


### 7.1.1 Filtry konwolucyjne

Rozważmy sieć neuronową, w której wejściem są obrazy reprezentowane przez piksele. Zakładamy na początek, że mamy jeden kanał i obraz jest w skali szarości. Na obrazie tym ustawiane jest w różnych położeniach prostokątne **okno** (*receptive field*), które zwykle ma postać kwadratu o rozmiarze $3\times 3$ lub $5\times 5$. Wewnątrz okna znajdują się (zwykle znormalizowane) piksele obrazu, które są przetwarzane przez **filtr konwolucyjny** za pomocą operacji **korelacji krzyżowej** (*cross-correlation*):

\begin{equation*}
z=\mathbf{w}^T\mathbf{x}+b,
\end{equation*}

gdzie $\mathbf{w}$ to wektor wag filtra, $b$ to bias, a $\mathbf{x}$ to wektor pikseli w oknie. Tablicę uzyskaną w wyniku zastosowania filtra $\mathbf{K}$ do wszystkich okien w obrazie $\mathbf{I}$ nazywamy **konwolucją** (*convolution*) i oznaczamy jako $\mathbf{I}\ast\mathbf{K}$. 

Zatem dla filtra o rozmiarze $h\times w$ mamy

\begin{equation*}
[\mathbf{I}\ast\mathbf{K}](i,j)=\sum_{m=0}^{h-1}\sum_{n=0}^{w-1} \mathbf{I}(i+m,j+n)\cdot \mathbf{K}(m,n),
\end{equation*}

gdzie $(i,j)$ to indeksy wartości w tablicy $\mathbf{I}\ast\mathbf{K}$.

Przykład: Dla $\mathbf{I}$ i $\mathbf{K}$ zdefiniowanych jako

\begin{equation*}
\mathbf{I}=\begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}, \quad
\mathbf{K}=\begin{bmatrix}
1 & 2\\
3 & 4
\end{bmatrix}
\end{equation*}

mamy

\begin{equation*}
\mathbf{I}\ast\mathbf{K}=\begin{bmatrix}
1\cdot 1+2\cdot 2+4\cdot 3+5\cdot 4 & 2\cdot 1+3\cdot 2+5\cdot 3+6\cdot 4\\
4\cdot 1+5\cdot 2+7\cdot 3+8\cdot 4 & 5\cdot 1+6\cdot 2+8\cdot 3+9\cdot 4
\end{bmatrix}=\begin{bmatrix}
1+4+12+20 & 2+6+15+24\\
4+10+21+32 & 5+12+24+36
\end{bmatrix}=\begin{bmatrix}
37 & 47\\
66 & 77
\end{bmatrix}.
\end{equation*}

Na wyjściu z warstwy konwolucyjnej znajduje się funkcja aktywacji, najczęściej ReLU, która jest stosowana do każdego elementu tablicy $\mathbf{I}\ast\mathbf{K}$. W ten sposób pojedyncze wyjście (neuron) uzyskuje postać:

\begin{equation*}
z=\text{ReLU}(\mathbf{w}^T\mathbf{x}+b).
\end{equation*}

Neuron $z$ "widzi" tylko tę część obrazu, która znajduje się w oknie, ale wagi wszystkich neuronów w warstwie konwolucyjnej są współdzielone. Funkcja ReLU wygasza te korelacje krzyżowe, które są mniejsze od $-b$ i pozostawia bez zmiany te, które są większe. Ponieważ $\mathbf{w}^T\mathbf{x}$ jest zwykłym iloczynem skalarnym, więc będzie przyjmował największe wartości dla tych okien, w których piksele są podobne (bliskie współliniowości) do wag filtra. W ten sposób neuron $z$ "wykrywa" lokalne cechy obrazu, które są reprezentowane przez wagi filtra $\mathbf{w}$.

Wagi filtra $\mathbf{w}$ są trenowane w procesie uczenia, a ich liczba jest znacznie mniejsza niż liczba wag w MLP. Na przykład, dla filtra o rozmiarze $3\times 3$ mamy 9 wag i 1 bias. 

Poniżej na przykładzie pokazuję w jaki sposób filtry konwolucyjne rozpoznają cechy lokalne w obrazach. 


Importy:

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image # do przetwarzania obrazów rastrowych

import torch
import torch.nn as nn

Pobieramy obrazek:

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Aptenodytes_forsteri_-Snow_Hill_Island%2C_Antarctica_-adults_and_juvenile-8.jpg/800px-Aptenodytes_forsteri_-Snow_Hill_Island%2C_Antarctica_-adults_and_juvenile-8.jpg"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
r = requests.get(url, headers=headers)
r.raise_for_status()

with open("penguin.jpg", "wb") as f: # Zapisz obraz do pliku
    f.write(r.content)

Wczytujemy obrazek za pomocą `PIL.Image` i nieco zmniejszamy:

In [ ]:
img = Image.open("penguin.jpg").convert("L")
img.thumbnail((img.height // 2, img.width // 2))
img

Konwersja do tablicy NumPy: obrazek rastrowy w skali szarości to tablica 2D o wymiarach (wysokość, szerokość) i zakresie wartości od 0 do 255. Każda wartość odpowiada intensywności piksela w danym miejscu - od 0 (czarny) do 255 (biały). 

In [ ]:
arr = np.array(img) # Konwersja obrazu do tablicy NumPy
arr

Dalsza konwersja, tym razem do tensora PyTorch. Wartości są normalizowane do zakresu [0, 1] i dodawany jest wymiar kanału (1 dla skali szarości) i wymiar wsadu (1, ponieważ mamy tylko jeden obrazek). W ten sposób uzyskujemy tensor o wymiarach (1, 1, wysokość, szerokość). Kanały i wsad to kwestie techniczne wymagane przez PyTorch. Na ogół jest tak, że PyTorch przetwarza dane w postaci wsadów, a nie pojedynczych próbek i w przypadku obrazów mają one zwykle wiele kanałów.

In [ ]:
arr = arr.astype(np.float32) / 255.0 # Normalizacja wartości pikseli do zakresu [0, 1]
img_tensor = torch.tensor(arr, dtype=torch.float32) # Konwersja do tensora PyTorch
img_tensor = img_tensor.unsqueeze(0).unsqueeze(0) # Dodanie wymiarów dla kanałów i wsadu
img_tensor.shape

Definiujemy filtr konwolucyjny do wykrywania krawędzi pionowych:

\begin{equation*}
\begin{bmatrix}
1 & 0 & -1 \\
1 & 0 & -1 \\
1 & 0 & -1
\end{bmatrix}
\end{equation*}

In [ ]:
vertical_filter = torch.tensor(
    [[1, 0, -1],
     [1, 0, -1],
     [1, 0, -1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0) 
vertical_filter

Wykonujemy konwolucję tensora obrazu z filtrem. Argument `padding=1` oznacza, że dodajemy jedną warstwę zer wokół obrazu, aby zachować jego rozmiar po konwolucji. Efektem operacji jest tensor o wymiarach (1, 1, wysokość, szerokość), który następnie przekształcamy do tablicy NumPy, normalizujemy z powrotem do zakresu [0, 255] i konwertujemy do formatu uint8. Na końcu przekształcamy tablicę NumPy na obrazek:

In [ ]:
img_conv = nn.functional.conv2d(img_tensor, vertical_filter, padding=1) # Konwolucja z filtrem pionowym
img_conv = img_conv.squeeze(0).squeeze(0).numpy() # Usunięcie wymiarów wsadu i kanałów
img_conv = (img_conv - img_conv.min()) / (img_conv.max() - img_conv.min()) * 255 # Ponowna normalizacja do zakresu [0, 255]
img_conv = img_conv.astype(np.uint8) # Konwersja do typu uint8
img_conv = Image.fromarray(img_conv) # Konwersja do obrazu
img_conv

**Ćwiczenie 7.1** 

Zdefiniuj filtr konwolucyjny do wykrywania krawędzi poziomych i wykonaj konwolucję na tym samym obrazku. Porównaj wyniki.

## Padding

Konwolucja obrazu o wymiarach $H\times W$ z filtrem o wymiarach $h\times w$ zmniejsza rozmiar obrazu zgodnie ze wzorem:

\begin{equation*}
H_{\text{out}}=H_{\text{in}}-h+1, \quad W_{\text{out}}=W_{\text{in}}-w+1.
\end{equation*}

*Padding*, czyli dopełnianie, to technika, która polega na dodaniu pikseli dookoła obrazu, aby uzyskać większy rozmiar wyjściowy. Dodanie $p$ pikseli dookoła obrazu o wymiarach $H\times W$ daje obraz o wymiarach $(H+2p)\times(W+2p)$. W przypadku konwolucji z filtrem o wymiarach $h\times w$ dostajemy finalnie tablicę o wymiarach:

\begin{equation*}
H_{\text{out}}=H_{\text{in}}+2p-h+1, \quad W_{\text{out}}=W_{\text{in}}+2p-w+1.
\end{equation*}

Dodatkowe piksele wypełnia się zwykle zerami, o ile wcześniej od obrazu odjęto średnią, dzięki czemu 0 jest wartością średnią pikseli. 


## Stride

*Stride* to krok, o jaki przesuwamy okno filtra po obrazie. Domyślnie jest równy 1, co oznacza, że filtr przesuwa się o jeden piksel w prawo lub w dół. Im większy krok, tym mniejszy rozmiar wyjściowy. Dla kroku $s$ i paddingu $p$ mamy:

\begin{equation*}
H_{\text{out}}=\left\lfloor\frac{H_{\text{in}}+2p-h}{s}\right\rfloor-1, \quad W_{\text{out}}=\left\lfloor\frac{W_{\text{in}}+2p-w}{s}\right\rfloor-1.
\end{equation*}

## Konwolucje z wieloma filtrami

Dla obrazu o $C$ kanałach filtr konwolucyjny ma wymiary $h\times w\times C$ i posiada $hwC$ wag plus bias. Aby sieć mogła wykrywać różnorodne cechy lokalne, filtry w warstwie organizuje się w tensor o wymiarach $h\times w\times C\times C_{\text{out}}$, gdzie $C_{\text{out}}$ to liczba filtrów w danej warstwie.

Łączna liczba parametrów w warstwie konwolucyjnej wynosi $(hwC+1)\cdot C_{\text{out}}$. **Kluczowa zaleta**: liczba parametrów nie zależy od rozmiaru obrazu ($H$ i $W$) - parametrów jest znacznie mniej niż w analogicznym zadaniu wymagałaby sieć MLP.

## Pooling

Pooling to operacja nie podobna konwolucji, która zmniejsza rozmiar obrazu. I w tym przypadku stosuje się prostokątne okno (*receptive field*), które przesuwa się po obrazie, nie ma jednak żadnych parametrów do wytrenowania. Efektem poolingu na pojedynczym oknie jest wynik działania prostej funkcji agregującej na pikselach tego okna, np. maksimum lub średniej. Krok przesuwania okna jest zwykle taki sam jak rozmiar okna, co oznacza, że nie nakładają się one na siebie. Pooling stosowany jest do każdego kanału z osobna, zmniejsza więc rozmiar obrazu, ale nie zmienia liczby kanałów.

## Equiwariantność i niezmienniczość

Sieci konwolucyjne są equiwariantne względem przesunięcia (**translation equivariant**), co oznacza, że przesunięcie cech w obrazie skutkuje odpowiednim przesunięciem cech w wyjściu. Equiwariantność jest zapewniana przez współdzielenie wag w warstwie konwolucyjnej - cecha zostanie wykryta niezależnie od tego, gdzie się znajduje w obrazie, i zostanie uwidoczniona w odpowiednim miejscu w mapie aktywacji.

Pooling powoduje, że lokalne niewielkie przesunięcia cech w obrazie nie mają wpływu na wyjście, ponieważ operacja ta wybiera dominujące wartości z małych regionów obrazu. W ten sposób sieci konwolucyjne stają się niezmiennicze (**translation invariant**) względem niewielkich przesunięć cech w obrazie.

## Architektura sieci konwolucyjnej

Typowa architektura prostej sieci konwolucyjnej składa się z następujących po sobie warstw:
1. konwolucyjna,
2. aktywacji (np. ReLU),
3. pooling (np. max pooling).

Sieć kończy się zwykle warstwą wypłaszczającą (*flatten*), która przekształca mapy aktywacji w wektory, po której następuje w pełni połączona sieć neuronowa (MLP), która przetwarza aktywacje z ostatniej warstwy konwolucyjnej. 

## [LeNet-5](http://en.wikipedia.org/wiki/LeNet-5)

Jedna z pierwszych sieci konwolucyjnych, zaprojektowana przez Yann LeCun i jego współpracowników w 1998 roku. Została zaprojektowana do rozpoznawania cyfr w obrazach o rozmiarze $28\times 28$ pikseli. 

Pobieramy zbiór MNIST. Zawiera on 60 000 obrazków cyfr odręcznych w skali szarości o rozmiarze $28\times 28$ pikseli w zbiorze uczącym i 10 000 analogicznych w zbiorze testowym. 

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import ToTensor, Normalize, Compose

mnist_transform = Compose([
    ToTensor(),
    Normalize(mean=[0.1307], std=[0.3081]),
])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=mnist_transform,
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=mnist_transform,
)

Tworzymy loadery do zbiorów uczącego i testowego. Używamy `shuffle=True`, aby tasować dane w przed każdą kolejną epoką treningu. 

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

Podgląd danych: 

In [ ]:
idx = np.random.randint(0, len(training_data), 6)
fig, axs = plt.subplots(2, 3, figsize=(10, 5))
for i, ax in enumerate(axs.flat):
    ax.imshow(training_data[idx[i]][0][0], cmap="gray")
    ax.axis("off")
    ax.set_title(f"Label: {training_data[idx[i]][1]}")
plt.tight_layout()

In [ ]:
class LeNetMNIST(nn.Module):
    
    def __init__(self):
        super(LeNetMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.activation = nn.Tanh()  
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.activation(self.conv1(x)) # --> 6x28x28
        x = self.pool(x) # --> 6x14x14
        x = self.activation(self.conv2(x)) # --> 16x10x10
        x = self.pool(x) # --> 16x5x5
        x = x.view(-1, 16 * 5 * 5) # Spłaszczenie do wektora
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = LeNetMNIST() # Inicjalizacja modelu
loss_fn = nn.CrossEntropyLoss() # Funkcja straty

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005) # Optymalizator

In [ ]:
n_epochs = 1
model.train()

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        # X: tensor o kształcie (batch_size, 1, 28, 28)
        # y: tensor o kształcie (batch_size,)
        # Predykcja i obliczenie straty na wsadzie
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Wykonanie kroku optymalizacji
        optimizer.zero_grad() # Zerowanie gradientów
        loss.backward() # Obliczanie gradientów
        optimizer.step() # Aktualizacja wag
        
        # Wyświetlenie straty co 100 wsadów
        if batch % 100 == 0:
            print(f"Batch {batch}, Loss: {loss.item():.4f}")
    print("Trening zakończony")

Przewidywania na zbiorze testowym:

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

y_true, y_pred = [], [] # Inicjalizacja list do przechowywania prawdziwych i przewidywanych etykiet

model.eval() # Ustawienie modelu w tryb ewaluacji

with torch.no_grad():
    for X, y in test_dataloader:
        pred = model(X)
        _, predicted = torch.max(pred, 1)
        y_true.extend(y.numpy())
        y_pred.extend(predicted.numpy())

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title(f"Macierz pomyłek (Dokładność: {acc:.4f})");

Indeksy obrazków, które zostały błędnie sklasyfikowane przez sieć

In [ ]:
# indeksy błędnych klasyfikacji
wrong_indices = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true != pred]
wrong_indices[:10] # pierwsze 10 błędnych klasyfikacji

Podgląd 6 losowo wybranych obrazków z błędnymi przewidywaniami:

In [ ]:
# 6 losowych błędnych klasyfikacji
fig, axs = plt.subplots(2, 3, figsize=(10, 5))
for i, ax in enumerate(axs.flat):
    idx = np.random.choice(wrong_indices)
    ax.imshow(test_data[idx][0][0], cmap="gray")
    ax.axis("off")
    ax.set_title(f"True: {test_data[idx][1]}, Pred: {y_pred[idx]}")
plt.tight_layout()